Kata del módulo 10

Si intentamos en un notebook, abrir un archivo inexistente sucede lo siguiente:

In [28]:
open("/path/to/mars.jpg")

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/mars.jpg'

Try y Except de los bloques
Vamos a usar el ejemplo de navegador a fin de crear código que abra archivos de configuración para la misión de Marte. Los archivos de configuración pueden tener todo tipo de problemas, por lo que es fundamental notificarlos con precisión cuando se presenten. Sabemos que, si no existe un archivo o directorio, se genera FileNotFoundError. Si queremos controlar esa excepción, podemos hacerlo con un bloque try y except:

In [29]:
try:
    open('config.txt')   
except FileNotFoundError:
    print("no se pudo encontrar el archivo 'config.txt'")

no se pudo encontrar el archivo 'config.txt'


Aunque es común un archivo que no existe, no es el único error que podemos encontrar. Los permisos de archivo no válidos pueden impedir la lectura de un archivo, incluso si este existe. Vamos a crear un archivo de Python denominado config.py. El archivo tiene código que busca y lee el archivo de configuración del sistema de navegación:

In [35]:
#Aquí se muestra que no se pudo manejar el error ya que no se contempla el "PermissionError" en el except
try:
     configuracion = open('config.txt')
except FileNotFoundError:
        print("Couldn't find the config.txt file!")

PermissionError: [Errno 13] Permission denied: 'config.txt'

Una manera poco útil de controlar este error sería detectar todas las excepciones posibles para evitar un traceback. Para comprender por qué detectar todas las excepciones es problemático, probaremos actualizando la función main():

In [33]:
  
try:       
    configuracion = open('config.txt')  
except Exception:
    print("No se encontró el archivo config.txt")

No se encontró el archivo config.txt


Vamos a corregir este fragmento de código para abordar todas estas frustraciones. Revertiremos la detección de FileNotFoundError y luego agregamos otro bloque except para detectar PermissionError:

In [41]:
#en el ejercicio se usa 'IsADirectoryError' en el segubdo except, pero yo usé 'PermissionError' ya que es la que me arroja a mi.
try:       
    configuracion = open('config.txt')  
except FileNotFoundError:
    print("No se encontró el archivo config.txt")
except PermissionError:
        print("No se tienen los permisos necesarios")

No se tienen los permisos necesarios


Cuando los errores son de una naturaleza similar y no es necesario controlarlos individualmente, puedes agrupar las excepciones como una usando paréntesis en la línea except. Por ejemplo, si el sistema de navegación está bajo cargas pesadas y el sistema de archivos está demasiado ocupado, tiene sentido detectar BlockingIOError y TimeOutError juntos:

In [43]:
# se pueden agrupar excepciones similares
try:
    configuration = open('config.txt')
except FileNotFoundError:
    print("Couldn't find the config.txt file!")
except IsADirectoryError:
    print("Found config.txt but it is a directory, couldn't read it")
except PermissionError:
        print("No se tienen los permisos necesarios")
except (BlockingIOError, TimeoutError):
    print("Filesystem under heavy load, can't complete reading configuration file")

No se tienen los permisos necesarios


Si necesitas acceder al error asociado a la excepción, debes actualizar la línea except para incluir la palabra clave as. Esta técnica es práctica si una excepción es demasiado genérica y el mensaje de error puede ser útil:

In [44]:
#puedes imprimir el error asosiado a la excepción
try:       
    configuracion = open('config.txt')  
except FileNotFoundError:
    print("No se encontró el archivo config.txt")
except PermissionError as error:
        print("No se tienen los permisos necesarios\n",error)

No se tienen los permisos necesarios
 [Errno 13] Permission denied: 'config.txt'


En este caso, as err significa que err se convierte en una variable con el objeto de excepción como valor. Después, usa este valor para imprimir el mensaje de error asociado a la excepción. Otra razón para usar esta técnica es acceder directamente a los atributos del error. Por ejemplo, si detecta una excepción OSError más genérica, que es la excepción primaria de FilenotFoundError y PermissionError, podemos diferenciarlas mediante el atributo .errno:

In [57]:
try:
    open("config.txt")
except OSError as error:
    if error.errno == 2:
        print("No se encontró el!")
    elif error.errno == 13:
        print("Se encontró config.txt pero no se pudo leer")

Se encontró config.txt pero no se pudo leer


Generación de excepciones

Los astronautas limitan su uso de agua a unos 11 litros al día. Vamos a crear una función que, con base al número de astronautas, pueda calcular la cantidad de agua quedará después de un día o más:

In [69]:
#función para estimar el agua restante
def water_left(astronauts, water_left, days_left):
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    return f"La cantidad de agua restante después de {days_left} días es: {total_water_left} litros"
#llamado a la función
water_left(5, 100, 2)

'La cantidad de agua restante después de 2 días es: -10 litros'

Esto no es muy útil, ya que una carencia en los litros sería un error. Después, el sistema de navegación podría alertar a los astronautas que no habrá suficiente agua para todos en dos días. Si eres un ingeniero(a) que programa el sistema de navegación, podrías generar una excepción en la función water_left() para alertar de la condición de error:

In [68]:
#se modifica la función para poder alertar la insuficiencia de agua
def water_left(astronauts, water_left, days_left):
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    if total_water_left < 0:
        raise RuntimeError(f"No hay suficiente agua para {astronauts} astronautas después de {days_left} días!")
    return f"La cantidad de agua restante después de {days_left} días es: {total_water_left} litros"
#llamado a la función
water_left(5, 100, 2)

RuntimeError: No hay suficiente agua para 5 astronautas después de 2 días!

La función water_left() también se puede actualizar para evitar el paso de tipos no admitidos. Intentenis pasar argumentos que no sean enteros para comprobar la salida de error:

In [62]:
#su prueba la función con tipos de datos no esperados
water_left("3", "200", None)

TypeError: can't multiply sequence by non-int of type 'NoneType'

El error de TypeError no es muy descriptivo en el contexto de lo que espera la función. Actualizaremos la función para que use TypeError, pero con un mensaje mejor:

In [67]:
def water_left(astronauts, water_left, days_left):
    for argument in [astronauts, water_left, days_left]:
        try:
            # su comprueba que el argumento sea de tipo int
            argument / 10
        except TypeError:
            # TypError será arrojado sólo si no es el tipo correcto de dato
            # Se arroja la misma excepción, pero se usa un mejor mensaje
            raise TypeError(f"Todos los argunmentos deben ser de tipo entero, modifique el argumento: '{argument}'")
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    if total_water_left < 0:
        raise RuntimeError(f"No hay suficiente agua para {astronauts} astronautas después de {days_left} días!")
    return f"La cantidad de agua restante después de {days_left} días es: {total_water_left} litros"

water_left("3", "200", None)

'La cantidad de agua restante después de 2 días es: 134 litros'